In [1]:
import pandas as pd
import numpy as np

Goal: create single-node version of Magude-Sede-Facazissa spatial sims, for e2e workflow

In [2]:
df_lookup = pd.read_csv("C:/Users/joshsu/Dropbox (IDM)/Malaria Team Folder/projects/mz_magude/inputs/csvs/reference/grid_lookup.csv")

In [3]:
df_lookup["catchment"].unique()

array(['Mapulanguene', 'Panjane-Caputine', 'Mahel', 'Chichuco', 'Moine',
       'Magude-Sede-Facazissa', 'Chicutso', 'Motaze'], dtype=object)

In [4]:
df_lookup

,grid_cell,lon,lat,bairro,bairro_name,catchment,pop
0,2,32.073248,-24.489278,79,Bairro ii,Mapulanguene,76.0
1,3,32.073248,-24.480295,80,Bairro iii,Mapulanguene,67.0
2,4,32.083268,-24.507244,79,Bairro ii,Mapulanguene,17.0
3,5,32.083268,-24.498261,79,Bairro ii,Mapulanguene,48.0
4,6,32.083268,-24.489278,79,Bairro ii,Mapulanguene,207.0
...,...,...,...,...,...,...,...
1183,1297,32.965009,-24.830638,5,Muazana,Motaze,48.0
1184,1298,32.965009,-24.821655,5,Muazana,Motaze,56.0
1185,1299,32.965009,-24.812671,5,Muazana,Motaze,15.0
1186,1300,32.975029,-24.830638,5,Muazana,Motaze,1.0


In [80]:
np.sum(df_lookup["pop"])

44529.0

In [5]:
df_itn = pd.read_csv("C:/Users/joshsu/Dropbox (IDM)/Malaria Team Folder/projects/mz_magude/inputs/csvs/interventions/grid_all_itn_events.csv")

In [6]:
df_itn = pd.merge(df_itn, df_lookup[["grid_cell", "pop"]], on="grid_cell", how="inner")

In [7]:
df_itn.columns

Index(['event', 'fulldate', 'cov_all', 'age_cov', 'min_season_cov',
       'fast_fraction', 'grid_cell', 'discard_type_flag', 'killing_initial',
       'killing_decay_constant', 'blocking_initial', 'blocking_decay_constant',
       'max_season_cov_day', 'pop'],
      dtype='object')

In [8]:
df_itn

,event,fulldate,cov_all,age_cov,min_season_cov,fast_fraction,grid_cell,discard_type_flag,killing_initial,killing_decay_constant,blocking_initial,blocking_decay_constant,max_season_cov_day,pop
0,1,2014-02-01,0.40,0.9,0.58,0.6,2,0,0.70,781,0.60,2046,40,76.0
1,2,2015-05-01,0.50,0.9,0.58,0.0,2,1,0.39,781,0.48,2046,40,76.0
2,3,2015-09-01,0.60,0.9,0.58,0.0,2,1,0.34,781,0.45,2046,40,76.0
3,4,2017-12-01,0.75,0.9,0.58,0.0,2,1,0.70,781,0.60,2046,40,76.0
4,1,2014-02-01,0.40,0.9,0.58,0.6,3,0,0.70,781,0.60,2046,40,67.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4191,4,2017-12-01,0.75,0.9,0.58,0.0,1299,1,0.70,781,0.60,2046,40,15.0
4192,1,2014-02-01,0.40,0.9,0.58,0.6,1301,0,0.70,781,0.60,2046,40,24.0
4193,2,2015-05-01,0.50,0.9,0.58,0.0,1301,1,0.39,781,0.48,2046,40,24.0
4194,3,2015-09-01,1.00,0.9,0.58,0.0,1301,1,0.34,781,0.45,2046,40,24.0


In [9]:
for i, sdf in df_itn.groupby("event"):
    print(sdf["fulldate"].unique())

['2014-02-01']
['2015-05-01']
['2015-09-01']
['2017-12-01']


In [10]:
def weighted_average(group, col_name, weight_column="pop"):
    return (group[col_name] * group[weight_column]).sum() / group[weight_column].sum()


In [11]:
from functools import partial

In [12]:
df_itn.groupby("event").apply(partial(weighted_average, col_name="cov_all"))

event
1    0.40000
2    0.50000
3    0.71559
4    0.75000
dtype: float64

In [13]:
df_itn.groupby("event").apply(partial(weighted_average, col_name="age_cov"))

event
1    0.9
2    0.9
3    0.9
4    0.9
dtype: float64

In [14]:
df_itn.groupby("event").apply(partial(weighted_average, col_name="discard_type_flag"))

event
1    0.0
2    1.0
3    1.0
4    1.0
dtype: float64

In [15]:
df_itn.groupby("event").apply(partial(weighted_average, col_name="fast_fraction"))

event
1    0.6
2    0.0
3    0.0
4    0.0
dtype: float64

In [16]:
df_itn.groupby("event").apply(partial(weighted_average, col_name="min_season_cov"))

event
1    0.58
2    0.58
3    0.58
4    0.58
dtype: float64

In [17]:
df_itn.groupby("event").apply(partial(weighted_average, col_name="killing_initial"))

event
1    0.70
2    0.39
3    0.34
4    0.70
dtype: float64

In [18]:
df_itn.groupby("event").apply(partial(weighted_average, col_name="blocking_initial"))

event
1    0.60
2    0.48
3    0.45
4    0.60
dtype: float64

In [19]:
df_hs = pd.read_csv("C:/Users/joshsu/Dropbox (IDM)/Malaria Team Folder/projects/mz_magude/inputs/csvs/interventions/grid_all_healthseek_events.csv")

In [20]:
df_hs = pd.merge(df_hs, df_lookup[["grid_cell", "pop"]], on="grid_cell", how="inner")

In [21]:
df_hs

,grid_cell,event,fulldate,duration,cov_newclin_youth,cov_newclin_adult,cov_severe_youth,cov_severe_adult,pop
0,2,1,2010-01-01,18250,0.892377,0.594918,0.892377,0.892377,76.0
1,3,1,2010-01-01,18250,0.892377,0.594918,0.892377,0.892377,67.0
2,4,1,2010-01-01,18250,0.887296,0.591530,0.887296,0.887296,17.0
3,5,1,2010-01-01,18250,0.892472,0.594982,0.892472,0.892472,48.0
4,6,1,2010-01-01,18250,0.894918,0.596612,0.894918,0.894918,207.0
...,...,...,...,...,...,...,...,...,...
1183,1297,1,2010-01-01,18250,0.747166,0.498110,0.800000,0.800000,48.0
1184,1298,1,2010-01-01,18250,0.736689,0.491126,0.800000,0.800000,56.0
1185,1299,1,2010-01-01,18250,0.747163,0.498108,0.800000,0.800000,15.0
1186,1300,1,2010-01-01,18250,0.164979,0.109986,0.800000,0.800000,1.0


In [22]:
df_hs.groupby("event").apply(partial(weighted_average, col_name="cov_newclin_youth"))

event
1    0.766945
dtype: float64

In [23]:
df_hs.groupby("event").apply(partial(weighted_average, col_name="cov_newclin_adult"))

event
1    0.511296
dtype: float64

In [24]:
df_hs.groupby("event").apply(partial(weighted_average, col_name="cov_severe_youth"))

event
1    0.834666
dtype: float64

In [25]:
np.sum(df_hs["pop"]*df_hs["cov_newclin_youth"])/np.sum(df_hs["pop"])

0.7669447140983027

In [26]:
df_irs = pd.read_csv("C:/Users/joshsu/Dropbox (IDM)/Malaria Team Folder/projects/mz_magude/inputs/csvs/interventions/grid_all_irs_events.csv")

In [27]:
df_irs = pd.merge(df_irs, df_lookup[["grid_cell", "pop"]], on="grid_cell", how="inner")

In [28]:
df_irs.groupby("fulldate").apply(partial(weighted_average, col_name="cov_all"))

fulldate
2014-09-01    0.550859
2015-09-01    0.817352
2016-09-01    0.692031
2017-09-01    0.806231
2017-10-01    0.730000
2017-11-01    0.330000
2017-12-01    0.860000
dtype: float64

In [29]:
df_irs.groupby("fulldate").apply(partial(weighted_average, col_name="killing"))

fulldate
2014-09-01    0.6
2015-09-01    0.9
2016-09-01    0.9
2017-09-01    0.9
2017-10-01    0.9
2017-11-01    0.9
2017-12-01    0.9
dtype: float64

In [30]:
df_irs.groupby("fulldate").apply(partial(weighted_average, col_name="box_duration"))

fulldate
2014-09-01      1.0
2015-09-01    210.0
2016-09-01    210.0
2017-09-01    210.0
2017-10-01    210.0
2017-11-01    210.0
2017-12-01    210.0
dtype: float64

In [31]:
df_irs.groupby("fulldate").apply(partial(weighted_average, col_name="exp_duration"))

fulldate
2014-09-01    90.0
2015-09-01    30.0
2016-09-01    30.0
2017-09-01    30.0
2017-10-01    30.0
2017-11-01    30.0
2017-12-01    30.0
dtype: float64

In [32]:
df_mda = pd.read_csv("C:/Users/joshsu/Dropbox (IDM)/Malaria Team Folder/projects/mz_magude/inputs/csvs/interventions/grid_all_mda_events.csv")

In [33]:
df_mda = pd.merge(df_mda, df_lookup[["grid_cell", "pop"]], on="grid_cell", how="inner")

In [34]:
for i, sdf in df_mda.groupby("event"):
    print(sdf["fulldate"].unique())

['2015-11-19' '2015-11-18' '2015-11-20' '2015-11-27' '2015-11-22'
 '2015-11-26' '2015-11-25' '2015-11-21' '2015-11-12' '2015-11-11'
 '2015-11-24' '2015-11-15' '2015-11-09' '2015-11-14' '2015-11-13'
 '2015-11-10' '2015-11-23' '2015-11-17' '2015-11-16']
['2016-01-27' '2016-01-28' '2016-01-26' '2016-01-31' '2016-02-04'
 '2016-01-29' '2016-02-02' '2016-01-30' '2016-01-20' '2016-01-21'
 '2016-01-23' '2016-01-14' '2016-01-22' '2016-01-19' '2016-02-01'
 '2016-01-18' '2016-01-25']
['2016-12-08' '2016-12-09' '2016-12-07' '2016-12-10' '2016-12-06'
 '2016-12-15' '2016-12-12' '2016-12-16' '2016-12-11' '2016-12-14'
 '2016-12-20' '2016-12-17' '2016-12-05' '2016-12-02' '2016-12-03'
 '2016-12-13' '2016-12-19']
['2017-01-27' '2017-01-28' '2017-01-30' '2017-02-08' '2017-01-25'
 '2017-02-01' '2017-02-04' '2017-02-09' '2017-02-07' '2017-02-06'
 '2017-02-02' '2017-01-26' '2017-01-31' '2017-01-24' '2017-01-21'
 '2017-01-20' '2017-01-23' '2017-01-17' '2017-01-19' '2017-01-18']


In [35]:
df_mda["fulldate"]

0       2015-11-19
1       2016-01-27
2       2016-12-08
3       2017-01-27
4       2015-11-19
           ...    
4739    2017-01-19
4740    2015-11-16
4741    2016-01-26
4742    2016-12-07
4743    2017-01-19
Name: fulldate, Length: 4744, dtype: object

In [36]:
from datetime import datetime

def date_to_ordinal(date_str):
    # Parse the input date string into a datetime object
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    
    # Get the ordinal date (day of the year) from the datetime object
    ordinal_date = date_obj.timetuple().tm_yday
    
    return ordinal_date


def ordinal_to_date(ordinal_date, year):
    # Create a datetime object for the given year and the ordinal date
    date_obj = datetime.strptime(f'{year}-{ordinal_date}', '%Y-%j')
    
    # Extract the date in "yyyy-mm-dd" format from the datetime object
    date_str = date_obj.strftime('%Y-%m-%d')
    
    return date_str


In [37]:
df_mda["date_ordinal"] = df_mda["fulldate"].apply(date_to_ordinal)

In [38]:
df_mda.groupby("event").apply(partial(weighted_average, col_name="date_ordinal")).round()

event
1    320.0
2     24.0
3    343.0
4     24.0
dtype: float64

In [39]:
ordinal_to_date(21,2010)

'2010-01-21'

In [40]:
ordinal_to_date(22,2010)

'2010-01-22'

In [41]:
ordinal_to_date(315,2010)

'2010-11-11'

In [42]:
ordinal_to_date(345,2010)

'2010-12-11'

In [43]:
for i, sdf in df_mda.groupby("event"):
    print(sdf["fulldate"].unique())

['2015-11-19' '2015-11-18' '2015-11-20' '2015-11-27' '2015-11-22'
 '2015-11-26' '2015-11-25' '2015-11-21' '2015-11-12' '2015-11-11'
 '2015-11-24' '2015-11-15' '2015-11-09' '2015-11-14' '2015-11-13'
 '2015-11-10' '2015-11-23' '2015-11-17' '2015-11-16']
['2016-01-27' '2016-01-28' '2016-01-26' '2016-01-31' '2016-02-04'
 '2016-01-29' '2016-02-02' '2016-01-30' '2016-01-20' '2016-01-21'
 '2016-01-23' '2016-01-14' '2016-01-22' '2016-01-19' '2016-02-01'
 '2016-01-18' '2016-01-25']
['2016-12-08' '2016-12-09' '2016-12-07' '2016-12-10' '2016-12-06'
 '2016-12-15' '2016-12-12' '2016-12-16' '2016-12-11' '2016-12-14'
 '2016-12-20' '2016-12-17' '2016-12-05' '2016-12-02' '2016-12-03'
 '2016-12-13' '2016-12-19']
['2017-01-27' '2017-01-28' '2017-01-30' '2017-02-08' '2017-01-25'
 '2017-02-01' '2017-02-04' '2017-02-09' '2017-02-07' '2017-02-06'
 '2017-02-02' '2017-01-26' '2017-01-31' '2017-01-24' '2017-01-21'
 '2017-01-20' '2017-01-23' '2017-01-17' '2017-01-19' '2017-01-18']


In [44]:
df_mda.groupby("event").apply(partial(weighted_average, col_name="cov_all"))

event
1    0.433804
2    0.282581
3    0.398330
4    0.396379
dtype: float64

In [45]:
df_rcd = pd.read_csv("C:/Users/joshsu/Dropbox (IDM)/Malaria Team Folder/projects/mz_magude/2021_model/inputs/grid_all_react_events_condensed.csv")

In [46]:
df_rcd = pd.merge(df_rcd, df_lookup[["grid_cell", "pop"]], on="grid_cell", how="inner")

In [47]:
df_rcd

,Unnamed: 0,index,grid_cell,event,fulldate,trigger_coverage,coverage,interval,pop
0,0,6,2,1,2017-06-01,0.500000,0.067509,-1,76.0
1,1,13,3,1,2017-06-01,0.500000,0.076577,-1,67.0
2,2,20,4,1,2017-06-01,0.500000,0.301805,-1,17.0
3,3,27,5,1,2017-06-01,0.500000,0.106889,-1,48.0
4,4,34,6,1,2017-06-01,0.500000,0.024786,-1,207.0
...,...,...,...,...,...,...,...,...,...
1181,1181,7700,1296,1,2017-06-01,0.576923,0.522056,-1,19.0
1182,1182,7707,1297,1,2017-06-01,0.576923,0.206647,-1,48.0
1183,1183,7714,1298,1,2017-06-01,0.576923,0.177126,-1,56.0
1184,1184,7721,1299,1,2017-06-01,0.576923,0.661271,-1,15.0


In [48]:
df_rcd.groupby("event").apply(partial(weighted_average, col_name="coverage"))

event
1    0.148667
dtype: float64

In [49]:
np.mean(df_rcd["pop"]*df_rcd["coverage"])

5.581546192474359

In [50]:
df_prev = pd.read_csv("C:/Users/joshsu/Dropbox (IDM)/Malaria Team Folder/projects/mz_magude/inputs/csvs/reference/grid_prevalence.csv")

In [51]:
df_prev = pd.merge(df_prev, df_lookup[["grid_cell", "pop"]], on="grid_cell", how="inner")

In [52]:
df_prev["N_pos"] = df_prev["prev"]*df_prev["N"]

In [53]:
for i, sdf in df_prev.groupby("round"):
    print(sdf["date"].unique())

['2015-06-08' '2015-06-09' '2015-06-10' '2015-06-12' '2015-06-05'
 '2015-06-04' '2015-06-06' '2015-05-21' '2015-06-11' '2015-06-07'
 '2015-05-23' '2015-05-27' '2015-05-25' '2015-05-26' '2015-05-22'
 '2015-05-24' '2015-05-20' '2015-05-29' '2015-05-30' '2015-05-18'
 '2015-05-17' '2015-05-16' '2015-05-15' '2015-05-19' '2015-06-01'
 '2015-05-28' '2015-05-14' '2015-06-02' '2015-06-03']
['2015-11-19' '2015-11-18' '2015-11-20' '2015-11-27' '2015-11-22'
 '2015-11-26' '2015-11-25' '2015-11-21' '2015-11-12' '2015-11-11'
 '2015-11-24' '2015-11-15' '2015-11-09' '2015-11-14' '2015-11-13'
 '2015-11-10' '2015-11-23' '2015-11-17' '2015-11-16']
['2016-01-27' '2016-01-28' '2016-01-29' '2016-02-02' '2016-01-30'
 '2016-02-04' '2016-01-21' '2016-01-20' '2016-01-23' '2016-01-22'
 '2016-01-31' '2016-01-19' '2016-01-18' '2016-01-25' '2016-02-01'
 '2016-01-26']
['2016-05-26' '2016-05-29' '2016-05-27' '2016-05-20' '2016-06-02'
 '2016-05-21' '2016-05-31' '2016-05-30' '2016-05-23' '2016-05-12'
 '2016-05-19' '2016

In [54]:
df_prev["date_ordinal"] = df_prev["date"].apply(date_to_ordinal)

In [55]:
df_prev.groupby("round").apply(partial(weighted_average, weight_column="N",col_name="date_ordinal")).round()

round
1    145.0
2    318.0
3     23.0
4    136.0
5    128.0
dtype: float64

In [56]:
ordinal_to_date(145,2010)

'2010-05-25'

In [74]:
ordinal_to_date(318,2010)

'2010-11-14'

In [75]:
ordinal_to_date(23,2010)

'2010-01-23'

In [77]:
ordinal_to_date(136,2016)

'2016-05-15'

In [78]:
ordinal_to_date(128,2017)

'2017-05-08'

In [57]:
df_prev.groupby("round").agg(N_sum=("N","sum"))

,N_sum
round,
1,1017
2,34680
3,5397
4,1536
5,3240


In [58]:
df_prev.groupby("round").agg(N_sum=("N_pos","sum"))

,N_sum
round,
1,90.697849
2,1302.000000
3,73.000000
4,18.000000
5,92.102517


In [59]:
df_prev.groupby("round").agg(N_sum=("N_pos","sum"))/df_prev.groupby("round").agg(N_sum=("N","sum"))

,N_sum
round,
1,0.089182
2,0.037543
3,0.013526
4,0.011719
5,0.028427


In [60]:
from statsmodels.stats.proportion import proportion_confint

In [64]:
proportion_confint(1,1000,method="beta")

(2.5317487491294045e-05, 0.005558924279826699)

In [73]:
import statsmodels.api as sm

def calculate_proportion_confint(df_count, df_nobs, alpha=0.05):
    """
    Calculate the confidence interval for the proportion based on count and nobs.

    Parameters:
        df_count (pandas.DataFrame): DataFrame with the count for each group (round).
        df_nobs (pandas.DataFrame): DataFrame with the number of observations (nobs) for each group (round).
        alpha (float, optional): The significance level for the confidence interval. Default is 0.05.

    Returns:
        pandas.DataFrame: DataFrame containing the lower and upper bounds of the confidence interval for each group.
    """
    df_result = df_count.copy()
    df_result['lower_bound'], df_result['upper_bound'] = zip(*df_result.apply(lambda row: sm.stats.proportion_confint(df_nobs.loc[row.name]['N_sum_pos'], row['N_sum'],  method="beta",alpha=alpha), axis=1))
    return df_result

# Example usage
df_prev_count = df_prev.groupby("round").agg(N_sum=("N", "sum"))
df_prev_nobs = df_prev.groupby("round").agg(N_sum_pos=("N_pos", "sum"))
confidence_interval_df = calculate_proportion_confint(df_prev_count, df_prev_nobs)
print(confidence_interval_df)

       N_sum  lower_bound  upper_bound
round                                 
1       1017     0.072383     0.108401
2      34680     0.035567     0.039597
3       5397     0.010617     0.016977
4       1536     0.006960     0.018458
5       3240     0.022979     0.034746


In [71]:
df_prev_count

,N_sum
round,
1,1017
2,34680
3,5397
4,1536
5,3240


In [68]:
df_prev_nobs

,N_sum_pos
round,
1,90.697849
2,1302.000000
3,73.000000
4,18.000000
5,92.102517


In [69]:
df_prev_nobs['N_sum_pos']

round
1      90.697849
2    1302.000000
3      73.000000
4      18.000000
5      92.102517
Name: N_sum_pos, dtype: float64

In [88]:
df_inc = pd.read_csv("C:/Users/joshsu/Dropbox (IDM)/Malaria Team Folder/projects/mz_magude/inputs/csvs/other/catchment_incidence_error_correction.csv")

In [89]:
df_inc = df_inc[df_inc["catchment"]=="Magude-Sede-Facazissa"]

In [90]:
df_inc

,catchment,cases,fulldate
0,Magude-Sede-Facazissa,1118,2014-01-01
7,Magude-Sede-Facazissa,557,2014-02-01
14,Magude-Sede-Facazissa,758,2014-03-01
21,Magude-Sede-Facazissa,250,2014-04-01
28,Magude-Sede-Facazissa,226,2014-05-01
35,Magude-Sede-Facazissa,238,2014-06-01
42,Magude-Sede-Facazissa,89,2014-07-01
49,Magude-Sede-Facazissa,56,2014-08-01
56,Magude-Sede-Facazissa,591,2014-09-01
63,Magude-Sede-Facazissa,508,2014-10-01


In [86]:
223*4.4

981.2

In [91]:
1118/223*10

50.134529147982065

In [92]:
df_lookup["pop"].sum()

44529.0

In [93]:
df_inc["pop"] = df_lookup["pop"].sum()

In [96]:
df_inc[["fulldate","cases","pop"]].to_csv("magude_incidence.csv", index=False)

In [97]:
df_inc

,catchment,cases,fulldate,pop
0,Magude-Sede-Facazissa,1118,2014-01-01,44529.0
7,Magude-Sede-Facazissa,557,2014-02-01,44529.0
14,Magude-Sede-Facazissa,758,2014-03-01,44529.0
21,Magude-Sede-Facazissa,250,2014-04-01,44529.0
28,Magude-Sede-Facazissa,226,2014-05-01,44529.0
35,Magude-Sede-Facazissa,238,2014-06-01,44529.0
42,Magude-Sede-Facazissa,89,2014-07-01,44529.0
49,Magude-Sede-Facazissa,56,2014-08-01,44529.0
56,Magude-Sede-Facazissa,591,2014-09-01,44529.0
63,Magude-Sede-Facazissa,508,2014-10-01,44529.0
